In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# Passo 1: Carregar e Preparar o Dataset
train_dir = 'train'
validation_dir = 'test'

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

# Passo 2: Criar e Treinar o Modelo
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3), name='conv2d_1'),
    tf.keras.layers.MaxPooling2D(2, 2, name='max_pooling2d_1'),
    tf.keras.layers.Flatten(name='flatten_1'),
    tf.keras.layers.Dense(1, activation='sigmoid', name='dense_1')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo (pelo menos 1 época para inicializar as camadas)
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=1,  # Treinar por pelo menos 1 época
    validation_data=validation_generator,
    validation_steps=50
)

# Passo 3: Criar o Modelo para Visualizar Ativações
# Usar os nomes corretos das camadas
last_conv_layer_name = 'conv2d_1'  # Última camada convolucional
last_deep_layer_name = 'dense_1'   # Última camada densa

# Criar o modelo intermediário
grad_model = Model(
    inputs=model.input,
    outputs=[
        model.get_layer(last_conv_layer_name).output,  # Saída da última camada convolucional
        model.get_layer(last_deep_layer_name).output    # Saída da última camada densa
    ]
)

# Passo 4: Obter e Visualizar as Ativações
img_path = '1.jpg'
img = image.load_img(img_path, target_size=(150, 150))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

# Obter as ativações
conv_output, dense_output = grad_model.predict(img_tensor)

# Função para visualizar as ativações
def display_activation(activations, col_size, row_size, act_index):
    activation = activations[act_index]
    activation_index = 0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5, col_size*1.5))
    for row in range(0, row_size):
        for col in range(0, col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index], cmap='viridis')
            activation_index += 1

# Exibir as ativações da última camada convolucional
display_activation([conv_output], 4, 8, 0)
plt.show()

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


c:\Users\Administrador\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Administrador\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.5504 - loss: 1.1763 - val_accuracy: 0.6430 - val_loss: 0.6526


AttributeError: The layer sequential has never been called and thus has no defined input.